In [1]:
#Building a regression model that tries to predict the estimated cumulative 6 month revenue from a new customer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import datetime as dt
%matplotlib inline
import seaborn as sns
import lifetimes

In [2]:
item = pd.read_csv('olist_order_items_dataset.csv')
customer = pd.read_csv('olist_customers_dataset.csv')
payment = pd.read_csv('olist_order_payments_dataset.csv')
review = pd.read_csv('olist_order_reviews_dataset.csv')	
order = pd.read_csv('olist_orders_dataset.csv')
product = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

In [3]:
#order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value
cols = ['order_id','product_id', 'seller_id', 'price', 'freight_value']
item = item[cols]
item = item.set_index('order_id')
item


,product_id,seller_id,price,freight_value
order_id,,,,
00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14
...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41
fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,350.00,36.53
fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95


In [4]:
item.shape

(112650, 4)

In [5]:
item[('seller_id')].describe()

count                               112650
unique                                3095
top       6560211a19b47992c3666cc44a7e94c0
freq                                  2033
Name: seller_id, dtype: object

In [6]:
item[('product_id')].describe()

count                               112650
unique                               32951
top       aca2eb7d00ea1a7b8ebd4e68314663af
freq                                   527
Name: product_id, dtype: object

In [7]:
cols =['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
payment = payment[cols]

payment

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [8]:
payment.shape

(103886, 5)

In [9]:
payment= payment.groupby('order_id').sum()
payment

,payment_sequential,payment_installments,payment_value
order_id,,,
00010242fe8c5a6d1ba2dd792cb16214,1,2,72.19
00018f77f2f0320c557190d7a144bdd3,1,3,259.83
000229ec398224ef6ca0657da4fc703e,1,5,216.87
00024acbcdf0a6daa1e931b038114c75,1,2,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,1,3,218.04
...,...,...,...
fffc94f6ce00a00581880bf54a75a037,1,1,343.40
fffcd46ef2263f404302a634eb57f7eb,1,1,386.53
fffce4705a9662cd70adb13d4a31832d,1,3,116.85


In [10]:
customer = pd.read_csv('olist_customers_dataset.csv')
cols = ['customer_id','customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
customer = customer.set_index('customer_id')
customer.head()


,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
customer_id,,,,
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [11]:
customer.shape

(99441, 4)

In [12]:
customer['customer_city'].value_counts()

sao paulo             15540
rio de janeiro         6882
belo horizonte         2773
brasilia               2131
curitiba               1521
                      ...  
tibau do sul              1
lajeado grande            1
sao carlos do ivai        1
pocao                     1
japi                      1
Name: customer_city, Length: 4119, dtype: int64

In [13]:
#order table 
cols = ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
order = order[cols]
order = order.set_index('order_id')
order.head()

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
order_id,,,,,,,
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [14]:
order.shape

(99441, 7)

In [15]:
cols = ['order_id','review_score', 'review_creation_date', 'review_answer_timestamp']
review1 = review[cols]
review1 = review.set_index('order_id')
review1

,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
order_id,,,,,,
73fc7af87114b39712e6da79b0a377eb,7bc2406110b926393aa56f80a40eba40,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
a548910a1c6147796b98fdf73dbeba33,80e641a11e56f04c1ad469d5645fdfde,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
f9e4b658b201a9f2ecdecbb34bed034b,228ce5500dc1d8e020d8d1322874b6f0,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
658677c97b385a9be170737859d3511b,e64fb393e7b32834bb789ff8bb30750e,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
8e6bfb81e283fa7e4f11123a3fb894f1,f7c4243c7fe1938f181bec41a392bdeb,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...
22ec9f0669f784db00fa86d035cf8602,f3897127253a9592a73be9bdfdf4ed7a,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
55d4004744368f5571d1f590031933e4,b3de70c89b1510c4cd3d0649fd302472,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
7725825d039fc1f0ceb7635e3f7d9206,1adeb9d84d72fe4e337617733eb85149,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [16]:
review = review.sort_values(by='order_id', ascending=False, na_position='first')
review= review.groupby('order_id').mean()
review

,review_score
order_id,
00010242fe8c5a6d1ba2dd792cb16214,5.0
00018f77f2f0320c557190d7a144bdd3,4.0
000229ec398224ef6ca0657da4fc703e,5.0
00024acbcdf0a6daa1e931b038114c75,4.0
00042b26cf59d7ce69dfabb4e55b4fd9,5.0
...,...
fffc94f6ce00a00581880bf54a75a037,5.0
fffcd46ef2263f404302a634eb57f7eb,5.0
fffce4705a9662cd70adb13d4a31832d,5.0


In [17]:
#Joining tables
result = pd.merge(item, payment,how='left', on= 'order_id').reset_index()
result = result.sort_values(by='order_id')
result=result.drop_duplicates(subset=None, keep= 'first', inplace=False)
result.shape

(102425, 8)

In [18]:
result.head()

,order_id,product_id,seller_id,price,freight_value,payment_sequential,payment_installments,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,1.0,2.0,72.19
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,1.0,3.0,259.83
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,1.0,5.0,216.87
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,1.0,2.0,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,1.0,3.0,218.04


In [19]:
#Joining tables
#item, payment, product, order, customer, review
result = pd.merge(result, product,how='inner', on= 'product_id').sort_values(by='order_id')

In [20]:
result.shape

(102425, 16)

In [21]:
result = pd.merge(result, order,how='inner', on= 'order_id')
result = result.sort_values(by='order_id')
result = result.loc[:,~result.columns.duplicated()]
#result.columns

In [22]:
result1= result.loc[:,~result.columns.duplicated()]
result1.set_index('customer_id')




,order_id,product_id,seller_id,price,freight_value,payment_sequential,payment_installments,payment_value,product_category_name,product_name_lenght,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
customer_id,,,,,,,,,,,,,,,,,,,,,
3ce436f183e68e07877b285a838db11a,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,1.0,2.0,72.19,cool_stuff,58.0,...,650.0,28.0,9.0,14.0,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
f6dd3ec061db4e3987629fe6b26e5cce,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,1.0,3.0,259.83,pet_shop,56.0,...,30000.0,50.0,30.0,40.0,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
6489ae5e4333f3693df5ad4372dab6d3,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,1.0,5.0,216.87,moveis_decoracao,59.0,...,3050.0,33.0,13.0,33.0,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
d4eb9395c8c0431ee92fce09860c5a06,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,1.0,2.0,25.78,perfumaria,42.0,...,200.0,16.0,10.0,15.0,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
58dbd0b2d70206bf40e62cd34e84d795,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,1.0,3.0,218.04,ferramentas_jardim,59.0,...,3750.0,35.0,40.0,30.0,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b51593916b4b8e0d6f66f2ae24f2673d,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41,1.0,1.0,343.40,utilidades_domesticas,43.0,...,10150.0,89.0,15.0,40.0,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18 00:00:00
84c5d4fbaf120aae381fad077416eaa0,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,350.00,36.53,1.0,1.0,386.53,informatica_acessorios,31.0,...,8950.0,45.0,26.0,38.0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01 00:00:00
29309aa813182aaddc9b259e31b870e6,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95,1.0,3.0,116.85,esporte_lazer,43.0,...,967.0,21.0,24.0,19.0,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10 00:00:00


In [23]:
customer.head()


,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
customer_id,,,,
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [24]:
result1 = pd.merge(result1, customer,how='inner', on= 'customer_id').set_index('customer_id')
result1

,order_id,product_id,seller_id,price,freight_value,payment_sequential,payment_installments,payment_value,product_category_name,product_name_lenght,...,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
customer_id,,,,,,,,,,,,,,,,,,,,,
3ce436f183e68e07877b285a838db11a,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,1.0,2.0,72.19,cool_stuff,58.0,...,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
f6dd3ec061db4e3987629fe6b26e5cce,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,1.0,3.0,259.83,pet_shop,56.0,...,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
6489ae5e4333f3693df5ad4372dab6d3,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,1.0,5.0,216.87,moveis_decoracao,59.0,...,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
d4eb9395c8c0431ee92fce09860c5a06,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,1.0,2.0,25.78,perfumaria,42.0,...,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
58dbd0b2d70206bf40e62cd34e84d795,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,1.0,3.0,218.04,ferramentas_jardim,59.0,...,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b51593916b4b8e0d6f66f2ae24f2673d,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41,1.0,1.0,343.40,utilidades_domesticas,43.0,...,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18 00:00:00,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA
84c5d4fbaf120aae381fad077416eaa0,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,350.00,36.53,1.0,1.0,386.53,informatica_acessorios,31.0,...,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01 00:00:00,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR
29309aa813182aaddc9b259e31b870e6,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95,1.0,3.0,116.85,esporte_lazer,43.0,...,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10 00:00:00,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP


In [25]:
result1.columns

Index(['order_id', 'product_id', 'seller_id', 'price', 'freight_value',
       'payment_sequential', 'payment_installments', 'payment_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state'],
      dtype='object')

In [26]:
result1.drop(columns=['customer_unique_id_y', 'customer_zip_code_prefix_y', 'customer_city_y',
       'customer_state_y', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'])





KeyError: "['customer_unique_id_y' 'customer_zip_code_prefix_y' 'customer_city_y'\n 'customer_state_y'] not found in axis"

In [ ]:
counts = result1[('customer_unique_id')].value_counts()
count=result1.drop_duplicates()
counts

In [ ]:
fig = plt.figure(figsize=(10,10)) # define plot area
ax = fig.gca() # define axis    
counts.plot.bar(ax = ax) # Use the plot.bar method on the counts data frame
ax.set_title('Number of products by type') # Give the plot a main title
ax.set_xlabel('Product_category_name') # Set text for the x axis
ax.set_ylabel('Number of items')# Set text for y axis

In [ ]:
result1.columns

In [ ]:
result1['order_purchase_timestamp'] = pd.to_datetime(result1['order_purchase_timestamp'])
result1['order_date'] = result1.order_purchase_timestamp.dt.date
result1['order_delivered_customer_date'] = pd.to_datetime(result1['order_delivered_customer_date'])
result1['order_estimated_delivery_date'] = pd.to_datetime(result1['order_estimated_delivery_date'])
result1['order_approved_at'] = pd.to_datetime(result1['order_approved_at'])

In [ ]:
result1['order_purchase_timestamp'].describe

In [ ]:
result1['customer_unique_id'].describe

In [ ]:
DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2018-01-07 15:00:00', '2018-01-07 16:00:00',
               '2018-01-07 17:00:00', '2018-01-07 18:00:00',
               '2018-01-07 19:00:00', '2018-01-07 20:00:00',
               '2018-01-07 21:00:00', '2018-01-07 22:00:00',
               '2018-01-07 23:00:00', '2018-01-08 00:00:00'],
              dtype='datetime64[ns]', length=169, freq='H')

In [ ]:

result1['order_date'] = pd.order_purchase_timestamp.dt.date
result1['order_date'] = pd.to_datetime(result1['order_date'])

#cols = ['customer_unique_id', 'order_date']
#elog = elog[cols]

#elog.columns = ['CUSTOMER_ID', 'ORDER_DATE']


#result1.info()
display(result1.sample(5))